<a href="https://colab.research.google.com/github/MarcelaMonteiroMontenegroGallo/Python/blob/master/Uber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Configurar Colab para rodar Spark**


In [1]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [19]:
# iniciar uma sessão local 
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql import SQLContext
sc = SparkSession.builder.master('local[*]').getOrCreate()

In [21]:
sqlContext = SQLContext(sc)

In [35]:
get_uber_file = '/content/uber.csv'

**init session**


In [32]:
spark = SparkSession \
            .builder \
            .appName("etl-uber-py") \
            .config("spark.sql.warehouse.dir") \
            .enableHiveSupport() \
            .getOrCreate()

In [33]:
 # show configured parameters
 print(SparkConf().getAll())

[('spark.master', 'local[*]'), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true'), ('spark.app.name', 'pyspark-shell')]


In [34]:
 # set log level
spark.sparkContext.setLogLevel("INFO")

In [36]:
df_uber = spark.read \
        .format("csv") \
        .option("inferSchema", "true") \
        .option("header", "true") \
        .csv(get_uber_file)

In [37]:
#print schema of dataframe
df_uber.printSchema()

root
 |-- dispatching_base_number: string (nullable = true)
 |-- date: string (nullable = true)
 |-- active_vehicles: integer (nullable = true)
 |-- trips: integer (nullable = true)



In [39]:
  # display data
  df_uber.show()

+-----------------------+--------+---------------+-----+
|dispatching_base_number|    date|active_vehicles|trips|
+-----------------------+--------+---------------+-----+
|                 B02512|1/1/2015|            190| 1132|
|                 B02765|1/1/2015|            225| 1765|
|                 B02764|1/1/2015|           3427|29421|
|                 B02682|1/1/2015|            945| 7679|
|                 B02617|1/1/2015|           1228| 9537|
|                 B02598|1/1/2015|            870| 6903|
|                 B02598|1/2/2015|            785| 4768|
|                 B02617|1/2/2015|           1137| 7065|
|                 B02512|1/2/2015|            175|  875|
|                 B02682|1/2/2015|            890| 5506|
|                 B02765|1/2/2015|            196| 1001|
|                 B02764|1/2/2015|           3147|19974|
|                 B02765|1/3/2015|            201| 1526|
|                 B02617|1/3/2015|           1188|10664|
|                 B02598|1/3/20

In [41]:
 # register df into sql engine
 df_uber.createOrReplaceTempView("uber")

Quantos são e quais são as bases de carro do Uber?


In [43]:
# RESPOSTA 1
df_rp1 = spark.sql("""SELECT DISTINCT dispatching_base_number
FROM uber""")

In [65]:
#Número de bases de carros do Uber
df_rp1.count()

6

In [45]:
#Bases de carros do Uber 
df_rp1.show()

+-----------------------+
|dispatching_base_number|
+-----------------------+
|                 B02512|
|                 B02598|
|                 B02682|
|                 B02765|
|                 B02617|
|                 B02764|
+-----------------------+



Qual o total de veículos que passaram pela base B02617?


In [66]:
# RESPOSTA 2
df_rp2 = spark.sql("""SELECT count(active_vehicles) AS total_veiculos
FROM uber
WHERE dispatching_base_number = 'B02617' """)

In [67]:
# Total de veículos que passaram pela base B02617
df_rp2.show()

+--------------+
|total_veiculos|
+--------------+
|            59|
+--------------+



Qual o total de corridas por bases? descrecente

In [63]:
# RESPOSTA 3
df_rp3 = spark.sql(""" SELECT dispatching_base_number,
       sum(trips)
FROM uber
GROUP BY 1
ORDER BY sum(trips) DESC
  """)

In [64]:
df_rp3.show()

+-----------------------+----------+
|dispatching_base_number|sum(trips)|
+-----------------------+----------+
|                 B02764|   1914449|
|                 B02617|    725025|
|                 B02682|    662509|
|                 B02598|    540791|
|                 B02765|    193670|
|                 B02512|     93786|
+-----------------------+----------+

